# Lab4: Multiword expressions identification and extraction

In [1]:
import collections
import locale
import math
import os
import spacy
import spacy.lang.pl
import spacy.tokenizer
import time

In [2]:
locale.setlocale(locale.LC_COLLATE, 'pl_PL.UTF-8')

'pl_PL.UTF-8'

In [3]:
ACT_DIRECTORY = '../files'

## Use SpaCy tokenizer API to tokenize the text from the law corpus.

Tokenizacja została wykonana przy pomocy kodu z poprzednich zajęć. Tokeny zostały przekonwertowane do samych małych liter oraz usunięte zostały białe znaki na początku i na końcu.

In [4]:
nlp = spacy.load('pl_core_news_sm')
# Create a Tokenizer with the default settings for Polish
# including punctuation rules and exceptions
tokenizer = nlp.Defaults.create_tokenizer(nlp)

In [5]:
tokens_per_file = {}

for root, _, files in os.walk(ACT_DIRECTORY):
    for file_name in files:
        path = os.path.join(root, file_name)
        with open(path, encoding='utf-8') as file:
            content = file.read()
            tokens = [
                token.text.lower().strip() 
                for token 
                in tokenizer(content)
            ]
            tokens_per_file[path] = [token for token in tokens if token != '']

## Compute bigram counts of downcased tokens. 
Given the sentence: "The quick brown fox jumps over the lazy dog.", the bigram counts are as follows:
* "the quick": 1
* "quick brown": 1
* "brown fox": 1
* ...
* "dog .": 1

Przygotowana została generyczna metoda do zliczania n-gramów.

In [6]:
def count_ngrams(tokens, n=2):
    ngram_dict = collections.defaultdict(int)
    for words in zip(*[tokens[offset:] for offset in (range(n))]):
        ngram_dict[' '.join(words)] += 1
    return ngram_dict

In [7]:
bigram_counts = collections.defaultdict(int)

for file, tokens in tokens_per_file.items():
    for bigram, count in count_ngrams(tokens).items():
        bigram_counts[bigram] += count

In [8]:
sorted(bigram_counts.items(), key=lambda bigram_count: -bigram_count[1])[:20]

[('art .', 84214),
 ('ust .', 53600),
 ('poz .', 45455),
 (', poz', 43395),
 ('. 1', 40043),
 ('- -', 36548),
 ('r .', 33165),
 ('w art', 32188),
 (', o', 30040),
 ('mowa w', 28577),
 ('. 2', 26811),
 ('w ust', 23584),
 ('. art', 23029),
 (', w', 22569),
 ('. nr', 21491),
 ('2 .', 21372),
 ('1 .', 21347),
 (', z', 19769),
 (') w', 18361),
 ('. 3', 17197)]

## Discard bigrams containing characters other than letters. Make sure that you discard the invalid entries after computing the bigram counts.

Do wykrycia znaków innych niż litery wykorzystana została metoda `isalpha()` klasy str.

In [9]:
bigram_counts = {
    bigram: count 
    for (bigram, count) 
    in bigram_counts.items()
    if all([token.isalpha() for token in bigram.split(' ')])
}

In [10]:
sorted(bigram_counts.items(), key=lambda bigram_count: -bigram_count[1])[:20]

[('w art', 32188),
 ('mowa w', 28577),
 ('w ust', 23584),
 ('o których', 13942),
 ('których mowa', 13915),
 ('otrzymuje brzmienie', 9604),
 ('z dnia', 9579),
 ('o którym', 9224),
 ('którym mowa', 9211),
 ('do spraw', 8731),
 ('i nr', 8464),
 ('dodaje się', 8235),
 ('w brzmieniu', 7329),
 ('w drodze', 7137),
 ('na podstawie', 6720),
 ('stosuje się', 6565),
 ('w przypadku', 6069),
 ('o której', 5537),
 ('której mowa', 5518),
 ('w zakresie', 5487)]

## Use pointwise mutual information to compute the measure for all pairs of words.

Najpierw policzone zostało występowanie unigramów.

In [11]:
unigram_counts = collections.defaultdict(int)

for file, tokens in tokens_per_file.items():
    for unigram, count in count_ngrams(tokens, n=1).items():
        unigram_counts[unigram] += count
        
unigram_counts = {
    unigram: count 
    for (unigram, count) 
    in unigram_counts.items()
    if unigram.isalpha()
}

In [12]:
sorted(unigram_counts.items(), key=lambda unigram_count: -unigram_count[1])[:20]

[('w', 202063),
 ('i', 90324),
 ('art', 84246),
 ('z', 82814),
 ('o', 65202),
 ('do', 60977),
 ('ust', 53686),
 ('na', 50805),
 ('lub', 46100),
 ('się', 46066),
 ('poz', 45481),
 ('nr', 45157),
 ('oraz', 33654),
 ('r', 33337),
 ('mowa', 28891),
 ('nie', 23069),
 ('przez', 21026),
 ('pkt', 19287),
 ('dnia', 18051),
 ('których', 17996)]

Następnie te informacje wykorzystane zostały do policzenia PMI w oparciu o wzory z Wikipedii.

In [13]:
total_unigram_count = sum(unigram_counts.values())    
total_bigram_count = sum(bigram_counts.values())    

# log2(P(x, y) / (P(x) * P(y)))
def pmi2(bigram, separator=' '):
    first_token, second_token = bigram.split(separator)
    first_token_p = unigram_counts[first_token] / total_unigram_count
    second_token_p = unigram_counts[second_token] / total_unigram_count
    bigram_p = bigram_counts[bigram] / total_bigram_count
    return math.log2(bigram_p / (first_token_p * second_token_p))

In [14]:
bigram_pmi = {
    bigram: pmi2(bigram)
    for (bigram, count)
    in bigram_counts.items()
}

In [15]:
list(bigram_pmi.items())[:10]

[('ustawa z', 4.379094558700088),
 ('z dnia', 4.909796997264667),
 ('o zmianie', 5.943648353608395),
 ('zmianie ustawy', 7.632285568189541),
 ('ustawy o', 2.997759971025661),
 ('o utworzeniu', 5.972490051142801),
 ('utworzeniu agencji', 9.655040545312778),
 ('agencji techniki', 8.103824919750567),
 ('techniki i', 3.2836566055921494),
 ('i technologii', 3.7168414515812525)]

## Sort the word pairs according to that measure in the descending order and determine top 10 entries.

In [16]:
sorted(bigram_pmi.items(), key=lambda bigram_pmi: -bigram_pmi[1])[:10]

[('znająca pjm', 22.161515861722478),
 ('przynęt zatrutych', 22.161515861722478),
 ('magazynkiem mieszczącym', 22.161515861722478),
 ('torowiskach zasp', 22.161515861722478),
 ('najmniejszy promień', 22.161515861722478),
 ('ręczny miotacz', 22.161515861722478),
 ('klifów nadmorskich', 22.161515861722478),
 ('głazów narzutowych', 22.161515861722478),
 ('fenolami lotnymi', 22.161515861722478),
 ('ostrzeżony strzałami', 22.161515861722478)]

## Filter bigrams with number of occurrences lower than 5. Determine top 10 entries for the remaining dataset (>=5 occurrences).

In [17]:
filtered_bigram_pmi = {
    bigram: pmi
    for (bigram, pmi)
    in bigram_pmi.items()
    if bigram_counts[bigram] >= 5
}

In [18]:
sorted(filtered_bigram_pmi.items(), key=lambda bigram_pmi: -bigram_pmi[1])[:10]

[('grzegorz schetyna', 19.839587766835116),
 ('obiegów chłodzących', 19.839587766835116),
 ('otworami wiertniczymi', 19.839587766835116),
 ('świeckie przygotowujące', 19.839587766835116),
 ('klęskami żywiołowymi', 19.839587766835116),
 ('środa wlkp', 19.839587766835116),
 ('obcowania płciowego', 19.839587766835116),
 ('nietykalność cielesną', 19.839587766835116),
 ('ręcznego miotacza', 19.839587766835116),
 ('młyny kulowe', 19.839587766835116)]

## Use log likelihood ratio (LLR) to compute the measure for all pairs of words.

Do wyliczenia LLR wykorzystana została [gotowa implementacja podlinkowana w zadaniu](https://github.com/tdunning/python-llr/blob/master/llr.py).

In [19]:
def llr_2x2(k11, k12, k21, k22):
    '''Special case of llr with a 2x2 table'''
    return 2 * (denormEntropy([k11+k12, k21+k22]) +
                denormEntropy([k11+k21, k12+k22]) -
                denormEntropy([k11, k12, k21, k22]))

def denormEntropy(counts):
    '''Computes the entropy of a list of counts scaled by the sum of the counts. If the inputs sum to one, this is just the normal definition of entropy'''
    counts = list(counts)
    total = float(sum(counts))
    # Note tricky way to avoid 0*log(0)
    return -sum([k * math.log(k/total + (k==0)) for k in counts])

Zliczone zostało wystąpienie poszczególnych tokenów jako kolejno pierwsze i drugie słowo w bigramie.

In [20]:
bigram_first_token_count = collections.defaultdict(int)
bigram_second_token_count = collections.defaultdict(int)

for bigram, count in bigram_counts.items():
    first_token, second_token = bigram.split(' ')
    bigram_first_token_count[first_token] += count
    bigram_second_token_count[second_token] += count

In [21]:
bigram_llr = {}

for bigram, count in bigram_counts.items():
    first_token, second_token = bigram.split(' ')
    k11 = count
    k12 = bigram_second_token_count[second_token] - count
    k21 = bigram_first_token_count[first_token] - count
    k22 = total_bigram_count - (k11 + k12 + k21)
    
    bigram_llr[bigram] = llr_2x2(k11, k12, k21, k22)

In [22]:
list(bigram_llr.items())[:10]

[('ustawa z', 5232.874111163081),
 ('z dnia', 49825.50661258155),
 ('o zmianie', 8652.599377553095),
 ('zmianie ustawy', 7737.8292541178525),
 ('ustawy o', 5771.08663096017),
 ('o utworzeniu', 961.7506792048225),
 ('utworzeniu agencji', 366.47751015823815),
 ('agencji techniki', 80.42621761053306),
 ('techniki i', 82.29314483352937),
 ('i technologii', 118.54199846053962)]

## Sort the word pairs according to that measure in the descending order and display top 10 entries.

In [23]:
sorted(bigram_llr.items(), key=lambda bigram_llr: -bigram_llr[1])[:10]

[('mowa w', 178294.93722869083),
 ('otrzymuje brzmienie', 118778.94002869757),
 ('których mowa', 115960.97862050205),
 ('w art', 114780.25651876233),
 ('o których', 93619.1286106814),
 ('w ust', 88051.17502383213),
 ('którym mowa', 74918.5584188212),
 ('dodaje się', 66931.82314493437),
 ('do spraw', 61239.29428788787),
 ('o którym', 59613.87079361593)]

## Compute trigram counts for the whole corpus and perform the same filtering.

Występiwanie trigramów zostało zliczone przy pomocy przygotowanej wcześniej metody.

In [24]:
trigram_counts = collections.defaultdict(int)

for file, tokens in tokens_per_file.items():
    for trigram, count in count_ngrams(tokens, n=3).items():
        trigram_counts[trigram] += count

In [25]:
sorted(trigram_counts.items(), key=lambda trigram_count: -trigram_count[1])[:20]

[(', poz .', 43373),
 ('- - -', 34646),
 ('w art .', 32178),
 ('w ust .', 23547),
 ('ust . 1', 23356),
 ('. art .', 23024),
 ('r . nr', 17914),
 ('_ _ _', 16213),
 ('. 1 .', 15693),
 ('. 2 .', 15339),
 ('o których mowa', 13914),
 ('których mowa w', 13864),
 (', o których', 13832),
 (': 1 )', 13530),
 ('mowa w ust', 13493),
 ('mowa w art', 12372),
 (') w art', 11761),
 ('ust . 2', 10756),
 ('. 3 .', 9698),
 ('otrzymuje brzmienie :', 9556)]

In [26]:
trigram_counts = {
    trigram: count 
    for (trigram, count) 
    in trigram_counts.items()
    if all([token.isalpha() for token in trigram.split(' ')])
}

In [27]:
sorted(trigram_counts.items(), key=lambda trigram_count: -trigram_count[1])[:20]

[('o których mowa', 13914),
 ('których mowa w', 13864),
 ('mowa w ust', 13493),
 ('mowa w art', 12372),
 ('o którym mowa', 9209),
 ('którym mowa w', 9187),
 ('o której mowa', 5518),
 ('której mowa w', 5495),
 ('w drodze rozporządzenia', 4691),
 ('właściwy do spraw', 4635),
 ('minister właściwy do', 4615),
 ('ustawie z dnia', 3662),
 ('w ustawie z', 3658),
 ('stosuje się odpowiednio', 3103),
 ('ustawy z dnia', 3076),
 ('zastępuje się wyrazami', 2940),
 ('dodaje się ust', 2767),
 ('wejścia w życie', 2361),
 ('dni od dnia', 2076),
 ('się następujące zmiany', 1808)]

## Use PMI (with 5 occurrence threshold) and LLR to compute top 10 results for the trigrams. Devise a method for computing the values, based on the results for bigrams.

### PMI

PMI zostało wyliczone przy pomocy wzoru analogicznego to przypadku bigramu.

In [28]:
total_trigram_count = sum(trigram_counts.values())    

# log2(P(x, y, z) / (P(x) * P(y) * P(z)))
def pmi3(trigram, separator=' '):
    first_token, second_token, third_token = trigram.split(separator)
    first_token_p = unigram_counts[first_token] / total_unigram_count
    second_token_p = unigram_counts[second_token] / total_unigram_count
    third_token_p = unigram_counts[third_token] / total_unigram_count
    trigram_p = trigram_counts[trigram] / total_trigram_count
    return math.log2(trigram_p / (first_token_p * second_token_p * third_token_p))

In [29]:
trigram_pmi = {
    trigram: pmi3(trigram)
    for (trigram, count)
    in trigram_counts.items()
}

sorted(trigram_pmi.items(), key=lambda trigram_pmi: -trigram_pmi[1])[:10]

[('ostrzeżony strzałami ostrzegawczymi', 44.23724557324892),
 ('przekazowi pieniężnemu towarzyszyły', 44.23724557324892),
 ('grzyba synchytrium endobioticum', 44.23724557324892),
 ('clavibacter michiganensis ssp', 44.23724557324892),
 ('krzewiące etos społecznikowski', 44.23724557324892),
 ('wniebowzięcia najświętszej maryi', 44.23724557324892),
 ('najświętszej maryi panny', 44.23724557324892),
 ('salus aegroti suprema', 44.23724557324892),
 ('aegroti suprema lex', 44.23724557324892),
 ('prawosławnym metropolitą warszawskim', 44.23724557324892)]

In [30]:
filtered_trigram_pmi = {
    trigram: pmi
    for (trigram, pmi)
    in trigram_pmi.items()
    if trigram_counts[trigram] >= 5
}

sorted(filtered_trigram_pmi.items(), key=lambda trigram_pmi: -trigram_pmi[1])[:10]

[('finałowego turnieju mistrzostw', 37.07737423647053),
 ('profilem zaufanym epuap', 36.83636613696674),
 ('cienką sierścią zwierzęcą', 36.777813954611624),
 ('przedwczesnego wyrębu drzewostanu', 36.652283072527766),
 ('turnieju mistrzostw europy', 36.31183949010756),
 ('potwierdzonym profilem zaufanym', 36.288878341664244),
 ('szybkiemu postępowi technicznemu', 36.16042997619809),
 ('piłce nożnej uefa', 36.14132115325039),
 ('centralnemu biuru antykorupcyjnemu', 36.12872111647076),
 ('wypalonym paliwem jądrowym', 35.938037554861644)]

### LLR

Przy wyliczaniu LLR pary słów występujących w trigramie zostały potraktowane analogicznie do tokenów w przypadku bigramów.

In [37]:
trigram_first_pair_count = collections.defaultdict(int)
trigram_second_pair_count = collections.defaultdict(int)

for trigram, count in trigram_counts.items():
    first_token, second_token, third_token = trigram.split(' ')
    
    first_pair = f'{first_token} {second_token}'
    second_pair = f'{second_token} {third_token}'
    
    trigram_first_pair_count[first_pair] += count
    trigram_second_pair_count[second_pair] += count

In [38]:
trigram_llr = {}

for trigram, count in trigram_counts.items():
    first_token, second_token, third_token = trigram.split(' ')
    
    first_pair = f'{first_token} {second_token}'
    second_pair = f'{second_token} {third_token}'
    
    k11 = count
    k12 = trigram_second_pair_count[second_pair] - count
    k21 = trigram_first_pair_count[first_pair] - count
    k22 = total_trigram_count - (k11 + k12 + k21)
    
    trigram_llr[trigram] = llr_2x2(k11, k12, k21, k22)

In [39]:
sorted(trigram_llr.items(), key=lambda trigram_llr: -trigram_llr[1])[:10]

[('o których mowa', 168955.57335261998),
 ('których mowa w', 128677.68434874358),
 ('o którym mowa', 119516.38820732615),
 ('mowa w ust', 112720.72532532585),
 ('mowa w art', 91866.20358409919),
 ('którym mowa w', 83294.26652510924),
 ('o której mowa', 77146.35895224831),
 ('minister właściwy do', 64516.70557209867),
 ('w drodze rozporządzenia', 58164.02565077676),
 ('właściwy do spraw', 52909.678427205945)]

## Create a table comparing the methods (separate table for bigrams and trigrams).

### Bigram

| #| PMI | PMI (>=5 wystąpień) | LLR |
|-|-|-|-|
|1|_znająca pjm_ | _grzegorz schetyna_|_mowa w_|
|2|_przynęt zatrutych_|_obiegów chłodzących_|_otrzymuje brzmienie_|
|3|_magazynkiem mieszczącym_|_otworami wiertniczymi_|_których mowa_|
|4|_torowiskach zasp_|_świeckie przygotowujące_|_w art_|
|5|_najmniejszy promień_|_klęskami żywiołowymi_|_o których_|
|6|_ręczny miotacz_|_środa wlkp_|_w ust_|
|7|_klifów nadmorskich_|_obcowania płciowego_|_którym mowa_|
|8|_głazów narzutowych_|_nietykalność cielesną_|_dodaje się_
|9|_fenolami lotnymi_|_ręcznego miotacza_|_do spraw_|
|10|_ostrzeżony strzałami_|_młyny kulowe_|_o którym_|

### Trigram

| #| PMI | PMI (>=5 wystąpień) | LLR |
|-|-|-|-|
|1|_ostrzeżony strzałami ostrzegawczymi_|_finałowego turnieju mistrzostw_|_o których mowa_|
|2|_przekazowi pieniężnemu towarzyszyły_|_profilem zaufanym epuap_|_których mowa w_|
|3|_grzyba synchytrium endobioticum_|_cienką sierścią zwierzęcą_|_o którym mowa_|
|4|_clavibacter michiganensis ssp_|_przedwczesnego wyrębu drzewostanu_|_mowa w ust_|
|5|_krzewiące etos społecznikowski_|_turnieju mistrzostw europy_|_mowa w art_|
|6|_wniebowzięcia najświętszej maryi_|_potwierdzonym profilem zaufanym_|_którym mowa w_|
|7|_najświętszej maryi panny_|_szybkiemu postępowi technicznemu_|_o której mowa_|
|8|_salus aegroti suprema_|_piłce nożnej uefa_|_minister właściwy do_|
|9|_aegroti suprema lex_|_centralnemu biuru antykorupcyjnemu_|_w drodze rozporządzenia_|
|10|_prawosławnym metropolitą warszawskim_|_wypalonym paliwem jądrowym_|_właściwy do spraw_|

## Answer the following questions:
### Why do we have to filter the bigrams, rather than the token sequence?
Gdybyśmy dokonali filtrowania na etapie tokenizacji, to później znaleźlibyśmy się w sytuacji, w której tokeny z niepowiązanych miejsc w tekście zostałyby zgrupowane jako n-gramy.
### Which measure (PMI, PMI with filtering, LLR) works better for the bigrams and which for the trigrams? What types of expressions are discovered by the methods?
Wydaje mi się, że w obydwu przypadkach ciekawsze zostały frazy znalezione przy pomocy PMI. W większości przypadków stanowią one spójne kolokacje (*nietykalność cielesną*, *klęskami żywiołowymi*, *centralnemu biuru antykorupcyjnemu*). Przy pomocy LLR znalezione zostały frazy mniej ciekawe pod względem samej semantyki (*o których mowa*, *w ust*, *o których*), choć oczywiście z nich również można wyciągnąć pewne wnioski. Odfiltrowanie rzadko występujących wyników PMI wydaje się dobrym pomysłem -- pomogło zlikwidować wyniki, w których słowa wydają się urwane (*torowiskach zasp*) czy ogólnie mniej zrozumiałe (*nająca pjm*), a także wszelkie sentencje łacińskie (*aegroti suprema lex*).
### Can you devise a different type of filtering that would yield better results?
Rozważyłabym filtrację *stop words* (choć w przypadku trigramów wydaje mi się, że warto byłoby zachować środkowe słowo ze względu na frazy typu *X orax Y*) lub ogólnie krótsze tokeny. Taka filtracja miałaby szczególnie duży wpływ na wyniki LLR.